In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('structured-unstructured')
sc = SparkContext(conf=conf)

24/04/02 16:28:11 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.8.26 instead (on interface en0)
24/04/02 16:28:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 16:28:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# 예를 들어 다음과 같이 데이터가 주어질 때 미국의 $2,000불 이상 가져오기 위한 방법은??
ticker = sc.parallelize([
    (1, ("Google", "USA")),
    (2, ("Netflix", "USA")),
    (3, ("Amazon", "USA")),
    (4, ("Samsung", "Korea")),
    (5, ("Kakao", "Korea"))
])

prices = sc.parallelize([
    (1, (2984, "USD")),
    (2, (645, "USD")),
    (3, (2984, "USD")),
    (4, (11234, "KRW")),
    (5, (1234512, "KRW")),
])

In [15]:
# 방법 1 : Join -> Filter
tickerPrice = ticker.join(prices)
tickerPrice.filter(lambda x : x[1][0][1] == "USA" and x[1][1][0] > 2000).collect()

[(1, (('Google', 'USA'), (2984, 'USD'))),
 (3, (('Amazon', 'USA'), (2984, 'USD')))]

In [16]:
# 방법 2 : Filter -> Join
filterticker = ticker.filter(lambda x : x[1][1] == "USA")
filterprices = prices.filter(lambda x : x[1][0] > 2000)
filterticker.join(filterprices).collect()

[(1, (('Google', 'USA'), (2984, 'USD'))),
 (3, (('Amazon', 'USA'), (2984, 'USD')))]

### 방법1과 방법2 중에 뭐가 더 효율 적일까?
- 방법2 셔플링을 최소화했기 때문에
- 매번 최적화 작업이 필요한가? 스파크가 알아서 해주면 좋겠음

### 데이터 구조
- Unstructured
    - 로그 파일
    - 이미지 
    - 동영상

- Semi Structured: 행과 열이 존재
    - CSV
    - JSON
    - XML

- Structured: 행과 열 + 스키마 존재
    - 데이터베이스

- RDD
    - 데이터의 구조를 모르기때문에 개발자 의존
    - 개발자마다 성능이 천차만별
- Structured Data
    - 어떤 테스크를 수행할 것 인지 정의만하면 자동으로 최적화
    - SparkSQL은 구조화 된 데이터를 다룰 수 있게 해줌